VacationPy

Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,utrik,11.2278,169.8474,82.38,83,66,20.67,MH,1699510965
1,1,morehead city,34.7229,-76.7260,67.98,97,0,3.44,US,1699510966
2,2,puerto ayora,-0.7393,-90.3518,75.54,95,96,8.01,EC,1699510966
3,3,adamstown,-25.0660,-130.1015,72.46,87,100,17.98,PN,1699510966
4,4,ribeira grande,38.5167,-28.7000,67.77,97,100,18.01,PT,1699510967


Step 1: Create a map that displays a point for every city in the city_data_df
DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    hover_cols=["City"],
    geo=True,
    tiles="OSM",
    title="City Locations",
    frame_width=800,
    frame_height=500,
    color="City",
    size = 70,
    alpha = 0.5
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

Step2:Narrow down the city_data_df DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df.dropna() 

filtered_cities = filtered_cities[(filtered_cities['Max Temp'] > 70) & (filtered_cities['Max Temp'] < 73)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,adamstown,-25.0660,-130.1015,72.46,87,100,17.98,PN,1699510966
94,94,ulladulla,-35.3500,150.4667,72.77,96,99,11.86,AU,1699510995
154,154,saa,4.3667,11.4500,71.08,98,99,1.45,CM,1699511015
202,202,santo antonio,-6.3106,-35.4789,72.28,87,2,10.65,BR,1699511029
205,205,nova sintra,14.8667,-24.7167,72.54,68,90,11.86,CV,1699511030
220,220,ciudad sabinas hidalgo,26.5000,-100.1667,71.67,78,60,2.59,MX,1699511034
235,235,tazacorte,28.6290,-17.9293,70.77,77,100,12.66,ES,1699510847
271,271,mournies,35.4764,24.0092,71.83,83,20,13.80,GR,1699511050
272,272,sao joao da barra,-21.6403,-41.0511,70.97,81,2,15.95,BR,1699511050
287,287,alaghsas,17.0187,8.0168,71.49,17,77,8.05,NE,1699511055


In [20]:
# Check if there are any null values in the DataFrame
any_nulls = filtered_cities.isnull().any().any()

# Print the result
print(f"Are there any null values in the DataFrame? {any_nulls}")


Are there any null values in the DataFrame? False


Step 3: Create a new DataFrame called hotel_df.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,87,
94,ulladulla,AU,-35.3500,150.4667,96,
154,saa,CM,4.3667,11.4500,98,
202,santo antonio,BR,-6.3106,-35.4789,87,
205,nova sintra,CV,14.8667,-24.7167,68,
220,ciudad sabinas hidalgo,MX,26.5000,-100.1667,78,
235,tazacorte,ES,28.6290,-17.9293,77,
271,mournies,GR,35.4764,24.0092,83,
272,sao joao da barra,BR,-21.6403,-41.0511,81,
287,alaghsas,NE,17.0187,8.0168,17,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {"categories": "accommodation.hotel", "limit": limit, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
ulladulla - nearest hotel: Hotel Marlin
saa - nearest hotel: Hôtel Sa'a
santo antonio - nearest hotel: Pousada Onça Puma
nova sintra - nearest hotel: Residência Ka Dencho
ciudad sabinas hidalgo - nearest hotel: La Fogata
tazacorte - nearest hotel: App Leyma
mournies - nearest hotel: Oasis
sao joao da barra - nearest hotel: Pousada Mediterrâneo
alaghsas - nearest hotel: Hôtel Zagado
mushie - nearest hotel: hôtel Mushi
yokadouma - nearest hotel: Hotel elephant
vilhena - nearest hotel: No hotel found
mazamari - nearest hotel: Hospedaje El Conquistador
port lincoln - nearest hotel: Boston Hotel
boende - nearest hotel: Hotel Reference
kutum - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
tanabe - nearest hotel: 下部ホテル
evansville - nearest hotel: Doubletree by Hilton Evansville
new norfolk - nearest hotel: Woodbridge on the Derwent


,City,Country,Lat,Lng,Humidity,Hotel Name
3,adamstown,PN,-25.0660,-130.1015,87,No hotel found
94,ulladulla,AU,-35.3500,150.4667,96,Hotel Marlin
154,saa,CM,4.3667,11.4500,98,Hôtel Sa'a
202,santo antonio,BR,-6.3106,-35.4789,87,Pousada Onça Puma
205,nova sintra,CV,14.8667,-24.7167,68,Residência Ka Dencho
220,ciudad sabinas hidalgo,MX,26.5000,-100.1667,78,La Fogata
235,tazacorte,ES,28.6290,-17.9293,77,App Leyma
271,mournies,GR,35.4764,24.0092,83,Oasis
272,sao joao da barra,BR,-21.6403,-41.0511,81,Pousada Mediterrâneo
287,alaghsas,NE,17.0187,8.0168,17,Hôtel Zagado


Step 5: Add the hotel name and the country as addional infornation in the hover message for each cith in the map.

In [23]:
%%capture --no-display

mp2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    hover_cols=["Country", "Hotel Name"],
    geo=True,
    tiles="OSM",
    title="City Locations",
    frame_width=800,
    frame_height=500,
    color="City",
    size = 70,
    alpha = 0.5
)

mp2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)